In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import Counter
import mlflow
import sklearn
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 

# local imports
from prepare import *
from evaluate import *

Using TensorFlow backend.
C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\site-packages\tensorflow\python\util\nest.py:1286: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  _pywrap_tensorflow.RegisterType("Mapping", _collections.Mapping)
C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\604

In [2]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
)
from catboost import CatBoostRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import (
    LogisticRegression,
    SGDRegressor,
    Ridge,
    LinearRegression,
    Lasso
)
import warnings

warnings.filterwarnings(
    "ignore"
)  # Ridge classifier throws some warnings about ill-conditioned matrix

# Try out balanced classes
reduced_train = pd.read_csv("reduce_train.csv")

# reduced_train = pd.read_csv('reduce_train.csv')
reduced_test = pd.read_csv("reduce_test.csv")
reduced_train.shape, reduced_test.shape

categoricals = ["session_title"]
cols_to_drop = ["game_session", "installation_id", "accuracy_group"]


In [54]:
# NN_model = Sequential()


# # The Input Layer :
# NN_model.add(Dense(128, kernel_initializer='normal', input_dim=887, activation='relu'))

# # The Hidden Layers :
# NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# NN_model.add(Dropout(0.2))

# NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# NN_model.add(Dropout(0.2))

# NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# NN_model.add(Dropout(0.2))

# # The Output Layer :
# NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# # Compile model
# NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])


## Setup Models

In [72]:
lgbm = LGBMRegressor()
rfr = RandomForestRegressor()
cbr = CatBoostRegressor(
    loss_function="RMSE",
    task_type="CPU",
    learning_rate=0.05,
    iterations=2000,
    od_type="Iter",
    early_stopping_rounds=500,
    random_seed=42,
    silent=True
)
gbr = GradientBoostingRegressor()
abr = AdaBoostRegressor()
lvr = LinearSVR()
lr = LinearRegression()
sgd = SGDRegressor()
rr = Ridge()

models = [lgbm, rfr, cbr, gbr, abr, lr, rr]

In [73]:
def my_metric(y_true, y_pred):
    y_pred = get_class_pred(y_pred, reduced_train)
    qwk = cohen_kappa_score(y_true, y_pred, weights='quadratic')
    return qwk

### Read in data, set up stack

In [74]:
from vecstack import stacking

mms = MinMaxScaler()

#  Read in and scale data

# If running for submission use these 3 lines:
# X_train = reduced_train.drop(cols_to_drop, axis=1, errors="ignore")
# y_train = reduced_train.accuracy_group
# X_test = reduced_test.drop(cols_to_drop, axis=1, errors="ignore")

# If testing locally, use these line:
X = reduced_train.drop(cols_to_drop, axis=1, errors='ignore')
y = reduced_train.accuracy_group
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train, X_test = mms.fit_transform(X_train), mms.transform(X_test)

In [75]:
S_train, S_test = stacking(
    models,
    X_train,
    y_train,
    X_test,
    regression=True,
    mode="oof_pred_bag",
    needs_proba=False,
    save_dir=None,
    metric=my_metric,
    n_folds=4,
    shuffle=True,
    random_state=42,
    verbose=2,
)

task:         [regression]
metric:       [my_metric]
mode:         [oof_pred_bag]
n_models:     [7]

model  0:     [LGBMRegressor]
    fold  0:  [0.54823004]
    fold  1:  [0.58125588]
    fold  2:  [0.56638074]
    fold  3:  [0.60012286]
    ----
    MEAN:     [0.57399738] + [0.01908662]
    FULL:     [0.57401346]

model  1:     [RandomForestRegressor]
    fold  0:  [0.52941405]
    fold  1:  [0.56150637]
    fold  2:  [0.55333476]
    fold  3:  [0.57706670]
    ----
    MEAN:     [0.55533047] + [0.01722098]
    FULL:     [0.55504913]

model  2:     [CatBoostRegressor]
    fold  0:  [0.54517329]
    fold  1:  [0.58548968]
    fold  2:  [0.57375155]
    fold  3:  [0.59923641]
    ----
    MEAN:     [0.57591273] + [0.01990789]
    FULL:     [0.57553276]

model  3:     [GradientBoostingRegressor]
    fold  0:  [0.54157711]
    fold  1:  [0.58072165]
    fold  2:  [0.56781895]
    fold  3:  [0.60189576]
    ----
    MEAN:     [0.57300337] + [0.02184508]
    FULL:     [0.57262821]

model  

In [37]:
model = RandomForestRegressor()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))


Final prediction score: [0.57873227]


In [38]:
model = LinearRegression()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.59527373]


In [39]:
model = LGBMRegressor()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.58448582]


In [40]:
model = Ridge()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.59509393]


In [41]:
model = LinearSVR()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.59419494]


In [42]:
model = AdaBoostRegressor()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.57953660]


In [44]:
model = CatBoostRegressor(silent=True)

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
#y_pred = get_class_pred(y_pred, reduced_test)
print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

Final prediction score: [0.59563333]


In [186]:
model = Ridge()

model = model.fit(S_train, y_train)
y_pred = model.predict(S_test)
y_pred = get_class_pred(y_pred, reduced_test)

print('Final prediction score: [%.8f]' % my_metric(y_test, y_pred))

def make_submission(preds, train_df, filename):
    preds = get_class_pred(preds, train_df)
    # assert len(preds)==1000
    sample = pd.read_csv("data/sample_submission.csv")
    submission = pd.DataFrame()
    submission["installation_id"] = sample["installation_id"]
    submission["accuracy_group"] = preds
    submission.to_csv(filename, index=False)
    return submission

#make_submission(y_pred, reduced_train, 'vs_preds5.csv')

,installation_id,accuracy_group
0,00abaee7,3
1,01242218,3
2,017c5718,3
3,01a44906,3
4,01bc6cb6,3
...,...,...
995,fee254cf,3
996,ff57e602,0
997,ffc73fb2,3
998,ffe00ca8,1


In [54]:
make_submission(final_preds, reduced_train)

,installation_id,accuracy_group
0,00abaee7,1
1,01242218,3
2,017c5718,0
3,01a44906,0
4,01bc6cb6,3
...,...,...
995,fee254cf,3
996,ff57e602,1
997,ffc73fb2,3
998,ffe00ca8,3


In [57]:
from sklearn.model_selection import (cross_val_score, train_test_split, 
                                     GridSearchCV, RandomizedSearchCV)
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

# Tune all

#### LGBM

In [62]:
# LGBM
est = LGBMRegressor(boosting='gbdt', n_jobs=-1, random_state=2018)

hyper_space = {'n_estimators': sp_randint(1000, 2500),
               'max_depth':  [4, 5, 8, -1],
               'num_leaves': [15, 31, 63, 127],
               'subsample': sp_uniform(0.6, 0.4),
               'colsample_bytree': sp_uniform(0.6, 0.4)}

rs = RandomizedSearchCV(est, hyper_space, n_iter=60, cv=4, 
                         verbose=1, random_state=42, n_jobs=-1)

rs_results = rs.fit(X_train, y_train)

Fitting 4 folds for each of 60 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed: 11.0min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 44.4min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 61.2min finished


In [63]:
print("BEST PARAMETERS: " + str(rs_results.best_params_))
print("BEST CV SCORE: " + str(rs_results.best_score_))

# Predict (after fitting RandomizedSearchCV is an estimator with best parameters)
y_pred = rs.predict(X_test)

# Score
score = my_metric(y_test, y_pred)
print("QWK SCORE ON TEST DATA: {}".format(score))


BEST PARAMETERS: {'colsample_bytree': 0.718509402281633, 'max_depth': 4, 'n_estimators': 1064, 'num_leaves': 15, 'subsample': 0.9947547746402069}
BEST CV SCORE: 0.33196345612555345
QWK SCORE ON TEST DATA: 0.5769342903813786


BEST PARAMETERS: {'colsample_bytree': 0.718509402281633, 'max_depth': 4, 'n_estimators': 1064, 'num_leaves': 15, 'subsample': 0.9947547746402069}
BEST CV SCORE: 0.33196345612555345
QWK SCORE ON TEST DATA: 0.5769342903813786

### Catboost

In [ ]:
# LGBM
est = CatBoostRegressor(boosting='gbdt', n_jobs=-1, random_state=2018)

hyper_space = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
          'ctr_border_count':[50,5,10,20,100,200],
          'thread_count':4}

rs = RandomizedSearchCV(est, hyper_space, n_iter=60, cv=4, 
                         verbose=1, random_state=42, n_jobs=-1)

rs_results = rs.fit(X_train, y_train)
params = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          'border_count':[32,5,10,20,50,100,200],
          'ctr_border_count':[50,5,10,20,100,200],
          'thread_count':4}